In [7]:
import copy

initial_bays = [[1,2], [], [4], []]
initial_crane_position = 0
initial_crane_container_held = 3
initial_cost = 0

# Created a function to print a clearer visualization
# of the state

# This visualization is rotated clockwise: right and left are
# visualized as down and up.

def print_state(state_and_time):
    state = state_and_time[0]
    for i in range(len(state[0])):
        if i == state[1]:
            print(str(state[0][i])+" <=["+str(state[2])+"]-")
        else:
            print(str(state[0][i]))
    print("Total cost: " +str(state_and_time[1]))
    print()

# Created an object with information about the initial state

initial_state = ([initial_bays, initial_crane_position, initial_crane_container_held], 0)

print_state(initial_state)


[1, 2] <=[3]-
[]
[4]
[]
Total cost: 0



In [ ]:
# Here we create a function that allows us to compute the next state we end up
# with if we apply an action to a state

def perform_action(state, action):
    new_state = copy.deepcopy(state[0])
    new_state_cost = state[1]
    cost = 0
    if action == "RIGHT":
        if new_state[1] >= (len(new_state[0]) -1):
            return None
        new_state[1] = int(new_state[1])+1
        cost = 1